In [1]:
import os
import numpy as np
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
cuda = torch.cuda.is_available()
cuda

True

In [2]:
trainset = torchvision.datasets.ImageFolder(root='train_data/medium', 
                                                       transform=torchvision.transforms.ToTensor())
trainset.__len__(), len(trainset.classes)

(822154, 2300)

In [3]:
evalset = torchvision.datasets.ImageFolder(root='validation_classification/medium', 
                                                       transform=torchvision.transforms.ToTensor())
evalset.__len__(), len(evalset.classes)

(4601, 2300)

In [4]:
train_loader_args = dict(shuffle=True, batch_size=128,drop_last=False, num_workers=2,pin_memory=True) if cuda\
                    else dict(shuffle=True,drop_last=False, batch_size=128)
train_loader = DataLoader(trainset, **train_loader_args)

eval_loader_args = dict(shuffle=True, batch_size=128,drop_last=False, num_workers=2,pin_memory=True) if cuda\
                    else dict(shuffle=True,drop_last=False, batch_size=128)
eval_loader = DataLoader(evalset, **eval_loader_args)

In [5]:
class IdentityBlock(nn.Module):

    def __init__(self, in_channels, out_channels, stride):
        super(IdentityBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        if in_channels != out_channels:
            self.shortcut=[]
            self.shortcut.append(nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=0, bias=False))
            self.shortcut.append(nn.BatchNorm2d(out_channels))
            self.shortcut = nn.Sequential(*self.shortcut)
        else:
            self.shortcut = nn.Identity()

    def forward(self, x):
        y = F.relu(self.bn1(self.conv1(x)), inplace=True)
        y = self.bn2(self.conv2(y)) 
        y += self.shortcut(x)
        y = F.relu(y, inplace=True)
        return y

In [6]:
class CenterLoss(nn.Module):
    """
    Args:
        num_classes (int): number of classes.
        feat_dim (int): feature dimension.
    """
    def __init__(self, num_classes, feat_dim, device=torch.device('cpu')):
        super(CenterLoss, self).__init__()
        self.num_classes = num_classes
        self.feat_dim = feat_dim
        self.device = device
        
        self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim).to(self.device))

    def forward(self, x, labels):
        """
        Args:
            x: feature matrix with shape (batch_size, feat_dim).
            labels: ground truth labels with shape (batch_size).
        """
        batch_size = x.size(0)
        distmat = torch.pow(x, 2).sum(dim=1, keepdim=True).expand(batch_size, self.num_classes) + \
                  torch.pow(self.centers, 2).sum(dim=1, keepdim=True).expand(self.num_classes, batch_size).t()
        distmat.addmm_(1, -2, x, self.centers.t())

        classes = torch.arange(self.num_classes).long().to(self.device)
        labels = labels.unsqueeze(1).expand(batch_size, self.num_classes)
        mask = labels.eq(classes.expand(batch_size, self.num_classes))

        dist = []
        for i in range(batch_size):
            value = distmat[i][mask[i]]
            value = value.clamp(min=1e-12, max=1e+12) # for numerical stability
            dist.append(value)
        dist = torch.cat(dist)
        loss = dist.mean()

        return loss

In [7]:
class Network(nn.Module):

    def __init__(self, feat_dim=2):
        super(Network, self).__init__()
        
        self.layers = []
        
        self.layers.append(nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3, padding=1, stride=1, bias=False))
        self.layers.append(nn.BatchNorm2d(128))
        self.layers.append(nn.ReLU(inplace=True))
        
        self.layers.append(IdentityBlock(in_channels=128, out_channels=128, stride=1))
        self.layers.append(IdentityBlock(in_channels=128, out_channels=128, stride=1))
       
        self.layers.append(IdentityBlock(in_channels=128, out_channels=256, stride=2))
        self.layers.append(IdentityBlock(in_channels=256, out_channels=256, stride=1))
        
        self.layers.append(IdentityBlock(in_channels=256, out_channels=512, stride=1))
        self.layers.append(IdentityBlock(in_channels=512, out_channels=512, stride=1))
        
        self.layers.append(IdentityBlock(in_channels=512, out_channels=1024, stride=2))
        self.layers.append(IdentityBlock(in_channels=1024, out_channels=1024, stride=1))

        self.layers = nn.Sequential(*self.layers)
        
        self.linear_label = nn.Linear(1024, 2300, bias=True)
        
        # For creating the embedding to be passed into the Center Loss criterion
        self.linear_closs = nn.Linear(1024, feat_dim, bias=True)
        self.relu_closs = nn.ReLU(inplace=True)
    
    def forward(self, x, evalMode=False):
        output = x
        output = self.layers(output)

        output = F.avg_pool2d(output, [output.size(2), output.size(3)], stride=1)
        output = output.reshape(output.shape[0], output.shape[1])
        
        label_output = self.linear_label(output)
        label_output = label_output/torch.norm(self.linear_label.weight, dim=1)
        
        # Create the feature embedding for the Center Loss
        closs_output = self.linear_closs(output)
        closs_output = self.relu_closs(closs_output)

        return closs_output, label_output, output

def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)

In [8]:
def train(model, data_loader, test_loader, task='Classification'):
    model.train()

    for epoch in range(numEpochs):
        avg_loss = 0.0
        for batch_num, (feats, labels) in enumerate(data_loader):
            feats, labels = feats.to(device), labels.to(device)
            
            optimizer_label.zero_grad()
            optimizer_closs.zero_grad()
            
            feature, outputs, embedding = model(feats)

            l_loss = criterion_label(outputs, labels.long())
            c_loss = criterion_closs(feature, labels.long())
            loss = l_loss + closs_weight * c_loss
            
            loss.backward()
            
            optimizer_label.step()
            # by doing so, weight_cent would not impact on the learning of centers
            for param in criterion_closs.parameters():
                param.grad.data *= (1. / closs_weight)
            optimizer_closs.step()

            print('batch completed ' + str(batch_num / 822154 * 128 * 100) + '% ', end='\r')
            
            avg_loss += loss.item()
            if batch_num % 100 == 99:
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/100))
                avg_loss = 0.0
            
            torch.cuda.empty_cache()
            del feats
            del labels
            del loss
        
        if task == 'Classification':
            val_loss, val_acc = test_classify_closs(model, test_loader)
            #train_loss, train_acc = test_classify_closs(model, data_loader)
            print('Val Loss: {:.4f}\tVal Accuracy: {:.4f}'.format(val_loss, val_acc))

def test_classify_closs(model, test_loader):
    model.eval()
    test_loss = []
    accuracy = 0
    total = 0

    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        feature, outputs, embedding = model(feats)
        
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        pred_labels = pred_labels.view(-1)
        
        l_loss = criterion_label(outputs, labels.long())
        c_loss = criterion_closs(feature, labels.long())
        loss = l_loss + closs_weight * c_loss
        
        accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
        test_loss.extend([loss.item()]*feats.size()[0])
        del feats
        del labels

    model.train()
    return np.mean(test_loss), accuracy/total

In [9]:
device = torch.device("cuda" if cuda else "cpu")

numEpochs = 5
closs_weight = 0.003
num_feats = 3
learningRate = 3e-4
lr_cent = 0.5
feat_dim = 2

cnn=Network(feat_dim)
cnn_dict = cnn.state_dict()
pretrained = torch.load('resnetnet_best.pth')
pretrained_dict = pretrained.state_dict()
cnn_dict.update(pretrained_dict) 
cnn.load_state_dict(cnn_dict)

criterion_label = nn.CrossEntropyLoss()
criterion_closs = CenterLoss(2300, feat_dim, device)
optimizer_label = torch.optim.Adam(cnn.parameters(),lr=learningRate)
optimizer_closs = torch.optim.SGD(criterion_closs.parameters(), lr=lr_cent)

In [ ]:
cnn.train()
cnn.to(device)
train(cnn, train_loader, eval_loader,task='verification')

In [56]:
class TestDataset(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        img = torchvision.transforms.ToTensor()(img)
        return img

img_sequence=[]

def parse_data(datadir):
    img_list = []
    for root, directories, filenames in os.walk(datadir):
        for filename in filenames:
            #img_sequence.append(filename)
            if filename.endswith('.jpg'):
                filei = os.path.join(root,filename)
                img_list.append(filei)
                #img_list.append(filei[24:])

    print('{}{}'.format('#Images', len(img_list)))
    return img_list

img_list = parse_data('test_verification/')

testset = TestDataset(img_list)
test_loader_args = dict(shuffle=False, batch_size=256, pin_memory=True, drop_last=False) 
test_loader = DataLoader(testset, **test_loader_args)

#Images169392


In [83]:
def verify(model, test_loader):
  
  with torch.no_grad():
    model.eval()
    model.to(device)
    
    verification = torch.FloatTensor().to(device)

    for batch_num, feats in enumerate(test_loader):
        feats = feats.to(device)
        outputs = model(feats)[2]
        
        verification = torch.cat((verification, outputs), dim=0)
        
    return verification

embedding = verify(cnn, test_loader)

In [84]:
embeddings={}
for index, i in enumerate(img_list):
    embeddings[i[18:]]=embedding[index].cpu().numpy()

In [85]:
from sklearn.metrics.pairwise import cosine_similarity

trial=[]
score=[]

with open('test_trials_verification_student.txt') as f:
    content = f.readlines()

for line in content:
    trial.append(line.strip())
    line=line.strip().split(' ')
    embedding_1 = embeddings[line[0]].reshape(1,embeddings[line[0]].shape[0])
    embedding_2 = embeddings[line[1]].reshape(1,embeddings[line[0]].shape[0])
    similarity = cosine_similarity(embedding_1, embedding_2)
    score.append(similarity[0][0])

In [86]:
import pandas as pd
result = pd.DataFrame()
result['trial'] = trial
result['score'] = score
result.to_csv('/home/ubuntu/zichenli_hw2_verification_3.csv', encoding='utf-8', index=False)

In [55]:
from sklearn.metrics import roc_auc_score
import sys
import pandas as pd

y_true=[]
y_score=[]

with open('validation_trials_verification.txt') as f:
    content = f.readlines()

for line in content:
    line=line.split(' ')
    y_true.append(float(line[2]))

with open('validation_verification.txt') as f:
    content2 = f.readlines()

for line in content2:
    line=line.split(' ')
    y_score.append(float(line[2]))


auc = roc_auc_score(y_true, y_score)
print (auc)

0.9223930009582997
